In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [ ]:
PATH = "../input/train/"

In [ ]:
df_raw = pd.read_csv(f'{PATH}Train.csv', low_memory=False, 
                     parse_dates=["saledate"])

In [ ]:
df_raw.sort_values(by='saledate', inplace=True)

In [ ]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [ ]:
df_raw.SalePrice = np.log(df_raw.SalePrice)

In [ ]:
add_datepart(df_raw, 'saledate')

In [ ]:
train_cats(df_raw)

In [ ]:
df_raw.UsageBand.cat.categories

In [ ]:
df_raw.UsageBand.cat.set_categories(['High', 'Medium', 'Low'], ordered=True, inplace=True)

In [ ]:
df, y, nas = proc_df(df_raw, 'SalePrice')

In [ ]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 12000  # same as Kaggle's test set size
n_trn = len(df)-n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
_, X_valid = split_vals(df, n_trn)
_, y_valid = split_vals(y, n_trn)

In [ ]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [ ]:
df_trn, y_trn, nas = proc_df(raw_train, 'SalePrice', subset=30000, na_dict=nas)
X_train, _ = split_vals(df_trn, 20000)
y_train, _ = split_vals(y_trn, 20000)

In [ ]:
m = RandomForestRegressor(n_estimators=100, n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

In [ ]:
preds = np.stack([t.predict(X_valid) for t in m.estimators_])
preds.shape

In [ ]:
plt.plot([metrics.r2_score(y_valid, np.mean(preds[:i+1], axis=0)) for i in range(100)]);

In [ ]:
X_train, y_train, nas = proc_df(raw_train, 'SalePrice')

In [ ]:
set_rf_samples(20000)

In [ ]:
m = RandomForestRegressor(n_estimators = 40, n_jobs=-1, oob_score=True)
%time m.fit(X_train, y_train)
print_score(m)